!pip install transformers==4.40.1 accelerate==0.30.0 bitsandbytes==0.43.1 datasets==2.19.0 vllm==0.4.1 openai==1.25.1 -qqq

# 8.4절 실습: LLM 서빙 프레임워크

In [1]:
import os
# root_dir = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
from dotenv import load_dotenv
load_dotenv('.huggingface_env2')
print(os.environ['HF_HOME'])

/home/hyunkoo/DATA/HDD8TB/GenAI/Study_LLM/huggingface_local_cache


## 예제 8.1. 실습에 사용할 데이터셋 준비

In [3]:
import torch
from datasets import load_dataset

def make_prompt(ddl, question, query=''):
    prompt = f"""당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question을 해결할 수 있는 SQL 쿼리를 생성하세요.

### DDL:
{ddl}

### Question:
{question}

### SQL:
{query}"""
    return prompt

dataset = load_dataset("shangrilar/ko_text2sql", "origin")['test']
dataset = dataset.to_pandas()

for idx, row in dataset.iterrows():
  prompt = make_prompt(row['context'], row['question'])
  dataset.loc[idx, 'prompt'] = prompt

## 예제 8.2. 모델과 토크나이저를 불러와 추론 파이프라인 준비

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_id = "shangrilar/yi-ko-6b-text2sql"
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(model_id)
hf_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/9.74k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.28M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

## 예제 8.3. 배치 크기에 따른 추론 시간 확인

In [4]:
import time
for batch_size in [1, 2, 4, 8, 16, 32]:
  start_time = time.time()
  hf_pipeline(dataset['prompt'].tolist(), max_new_tokens=128, batch_size=batch_size)
  print(f'{batch_size}: {time.time() - start_time}')

1: 104.76487517356873
2: 137.6186866760254
4: 84.07006120681763
8: 53.870291233062744
16: 35.079726219177246
32: 27.54612112045288


## 예제 8.4. vLLM 모델 불러오기

### 안내
모델을 여러 번 GPU에 올리기 때문에 CUDA out of memory 에러가 발생할 수 있습니다. 그런 경우 구글 코랩의 런타임 > 세션 다시 시작 후 예제 코드를 실행해주세요.
예제 실행에 데이터셋이 필요한 경우 예제 8.1의 코드를 실행해주세요.

In [1]:
import torch
from vllm import LLM, SamplingParams

model_id = "shangrilar/yi-ko-6b-text2sql"
llm = LLM(model=model_id, dtype=torch.float16, max_model_len=1024)
# llm = LLM(model=model_id, max_model_len=1024)

INFO 08-27 19:25:26 llm_engine.py:176] Initializing an LLM engine (v0.5.3.post1) with config: model='shangrilar/yi-ko-6b-text2sql', speculative_config=None, tokenizer='shangrilar/yi-ko-6b-text2sql', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=1024, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None), seed=0, served_model_name=shangrilar/yi-ko-6b-text2sql, use_v2_block_manager=False, enable_prefix_caching=False)
INFO 08-27 19:25:26 model_runner.py:680] Starting to load model shangrilar/yi-ko-6b-text2sql...
INFO 08-27 19:25:27 weigh

Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]


INFO 08-27 19:25:29 model_runner.py:692] Loading model weights took 11.5127 GB
INFO 08-27 19:25:29 gpu_executor.py:102] # GPU blocks: 9159, # CPU blocks: 4096
INFO 08-27 19:25:30 model_runner.py:980] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 08-27 19:25:30 model_runner.py:984] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 08-27 19:25:35 model_runner.py:1181] Graph capturing finished in 5 secs.


## 예제 8.5. vLLM을 활용한 오프라인 추론 시간 측정

In [4]:
import time

for max_num_seqs in [1, 2, 4, 8, 16, 32]:
  start_time = time.time()
  llm.llm_engine.scheduler_config.max_num_seqs = max_num_seqs
  sampling_params = SamplingParams(temperature=1, top_p=1, max_tokens=128)
  outputs = llm.generate(dataset['prompt'].tolist(), sampling_params)
  print(f'{max_num_seqs}: {time.time() - start_time}')

Processed prompts: 100%|██████████| 112/112 [01:07<00:00,  1.66it/s, est. speed input: 326.30 toks/s, output: 64.12 toks/s]


1: 67.54550695419312


Processed prompts: 100%|██████████| 112/112 [00:35<00:00,  3.19it/s, est. speed input: 627.28 toks/s, output: 124.45 toks/s]


2: 35.13730192184448


Processed prompts: 100%|██████████| 112/112 [00:18<00:00,  5.95it/s, est. speed input: 1171.04 toks/s, output: 222.77 toks/s]


...

8: 12.624985933303833


Processed prompts: 100%|██████████| 112/112 [00:08<00:00, 13.46it/s, est. speed input: 2648.15 toks/s, output: 518.07 toks/s]


16: 8.335227489471436


Processed prompts: 100%|██████████| 112/112 [00:06<00:00, 17.68it/s, est. speed input: 3478.61 toks/s, output: 721.11 toks/s]

32: 6.349210262298584


## 예제 8.6. 온라인 서빙을 위한 vLLM API 서버 실행

### 안내
모델을 여러 번 GPU에 올리기 때문에 CUDA out of memory 에러가 발생할 수 있습니다. 그런 경우 구글 코랩의 런타임 > 세션 다시 시작 후 예제 코드를 실행해주세요.
예제 실행에 데이터셋이 필요한 경우 예제 8.1의 코드를 실행해주세요.

In [ ]:
!python -m vllm.entrypoints.openai.api_server \
--model shangrilar/yi-ko-6b-text2sql --host 127.0.0.1 --port 8888 --max-model-len 1024

## 예제 8.7. 백그라운드에서 vLLM API 서버 실행하기

In [6]:
!nohup python -m vllm.entrypoints.openai.api_server \
--model shangrilar/yi-ko-6b-text2sql --host 127.0.0.1 --port 8888 --max-model-len 512 &

nohup: appending output to 'nohup.out'


## 예제 8.8. API 서버 실행 확인

In [9]:
!curl http://localhost:8888/v1/models

curl: (7) Failed to connect to localhost port 8888 after 0 ms: Connection refused


## 예제 8.9. API 요청

In [ ]:
import json

json_data = json.dumps(
    {"model": "shangrilar/yi-ko-6b-text2sql",
      "prompt": dataset.loc[0, "prompt"],
      "max_tokens": 128,
      "temperature": 1}
    )

!curl http://localhost:8888/v1/completions \
    -H "Content-Type: application/json" \
    -d '{json_data}'

## 예제 8.10. OpenAI 클라이언트를 사용한 API 요청

In [ ]:
from openai import OpenAI

openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8888/v1"
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)
completion = client.completions.create(model="shangrilar/yi-ko-6b-text2sql",
                                 prompt=dataset.loc[0, 'prompt'], max_tokens=128)
print("생성 결과:", completion.choices[0].text)